In [2]:
import numpy as np
import sympy
from differentialGeometry3 import computeGeometry, printGeometry, determinantg, ginv
from IPython.display import display, Math

def displayNonZero(T):
    """Displays the non-zero components of the tensor T"""
    for index, element in np.ndenumerate(T):
        if element != 0:
            print(index)
            display(sympy.simplify(element))

**b)**

Finding the non-zero contributions to the Ricci tensor.
Note that we use the `differentialGeometry3` library from http://web.phys.ntnu.no/~mika/QF/software.html.

In [3]:
t, r, theta, phi = sympy.symbols('t r theta phi')
coords = np.array([t, r, theta, phi])
A = sympy.Function('A')(r)
B = sympy.Function('B')(r)

# Construct the tensor
g_ = np.zeros((4, 4),dtype=object)
g_[0,0] = A
g_[1,1] = -B
g_[2,2] = -r**2
g_[3,3] = -r**2 * sympy.sin(theta)**2

geometry = computeGeometry(g_,coords)

Covariant derivatives of covariant metric vanishes: True
Covariant derivatives of contravariant metric vanishes: True
Covariant divergence of the Einstein tensor vanishes: True


In [4]:
printGeometry(geometry)


ScalarCurvature = -Derivative(A(r), (r, 2))/(A(r)*B(r)) + Derivative(A(r), r)*Derivative(B(r), r)/(2*A(r)*B(r)**2) + Derivative(A(r), r)**2/(2*A(r)**2*B(r)) + 2*Derivative(B(r), r)/(r*B(r)**2) - 2*Derivative(A(r), r)/(r*A(r)*B(r)) + 2/r**2 - 2/(r**2*B(r))

Nonzero components of Christoffel symbols:
C^0_01 = Derivative(A(r), r)/(2*A(r))
C^1_00 = Derivative(A(r), r)/(2*B(r))
C^1_11 = Derivative(B(r), r)/(2*B(r))
C^1_22 = -r/B(r)
C^1_33 = -r*sin(theta)**2/B(r)
C^2_12 = 1/r
C^2_33 = -sin(theta)*cos(theta)
C^3_13 = 1/r
C^3_23 = cos(theta)/sin(theta)

Nonzero components of Einstein tensor:
G^00 = -(r*Derivative(B(r), r) + B(r)**2 - B(r))/(r**2*A(r)*B(r)**2)
G^11 = (-Derivative(A(r), r)/(r*A(r)) + B(r)/r**2 - 1/r**2)/B(r)**2
G^22 = (-2*r*A(r)*B(r)*Derivative(A(r), (r, 2)) + r*A(r)*Derivative(A(r), r)*Derivative(B(r), r) + r*B(r)*Derivative(A(r), r)**2 + 2*A(r)**2*Derivative(B(r), r) - 2*A(r)*B(r)*Derivative(A(r), r))/(4*r**3*A(r)**2*B(r)**2)
G^33 = (-2*r*A(r)*B(r)*Derivative(A(r), (r, 2)) + 

In [5]:
# Ricci
r4i_ = geometry[7]

# Ricci is diagonal
for ele in np.diag(r4i_):
    display(sympy.simplify(ele))
    print("----")

-Derivative(A(r), (r, 2))/(2*B(r)) + Derivative(A(r), r)*Derivative(B(r), r)/(4*B(r)**2) + Derivative(A(r), r)**2/(4*A(r)*B(r)) - Derivative(A(r), r)/(r*B(r))

----


Derivative(A(r), (r, 2))/(2*A(r)) - Derivative(A(r), r)*Derivative(B(r), r)/(4*A(r)*B(r)) - Derivative(A(r), r)**2/(4*A(r)**2) - Derivative(B(r), r)/(r*B(r))

----


-r*Derivative(B(r), r)/(2*B(r)**2) + r*Derivative(A(r), r)/(2*A(r)*B(r)) - 1 + 1/B(r)

----


(-r*A(r)*Derivative(B(r), r) + r*B(r)*Derivative(A(r), r) - 2*(B(r) - 1)*A(r)*B(r))*sin(theta)**2/(2*A(r)*B(r)**2)

----


Which is exactly what we expect.

**c)**

We need the determinant of our metric for the calculations.

In [6]:
determinantg(g_)

-r**4*A(r)*B(r)*sin(theta)**2

In [7]:
display(g_)

array([[A(r), 0, 0, 0],
       [0, -B(r), 0, 0],
       [0, 0, -r**2, 0],
       [0, 0, 0, -r**2*sin(theta)**2]], dtype=object)

**d)**

Let us calculate the stress tensor.
It is defined as

$$
T^{\mu \nu} = -F^{\mu \alpha} F^{\nu}_{\alpha} + \frac14 g^{\mu \nu} F_{\alpha \beta}F^{\alpha \beta}.
$$
We define the function `stressTensor` which calculates this.
Note that we use matrix-operations provided by NumPy.

In [8]:
def stressTensor(F, g_):
    """Compute stress tensor T from EM-field F"""
    g = ginv(g_)
    d = g_[:,0].size
    T = np.zeros((d,d),dtype=object)
    for mu in range(d):
        for nu in range(d):
            T[mu, nu] = - F[mu, :] @ (g_ @ F)[:, nu] + sympy.Rational(1, 4) * g[mu, nu] * np.tensordot(g_ @ F @ g_, F)
    
    return T

Now, we want the stress tensor with lower indices.
That is
$$
T_{\mu \nu} = g_{\mu \sigma} T^{\sigma \gamma} g_{\gamma \nu}
$$

In [9]:
def lower(T):
    """Lower the indices of the (upper) tensor T"""
    return g_ @ T @ g_

In [10]:
F = np.zeros(g_.shape, dtype=object)
Q = sympy.symbols('Q')
E = sympy.sqrt(A * B) * Q / (4*sympy.pi*r**2)

F[1, 0] = -E / (A * B)
F[0, 1] = E / (A * B)


T = stressTensor(F, g_)

In [11]:
T_ = sympy.simplify(lower(T))
displayNonZero(T_)

(0, 0)


-3*Q**2*A(r)/(32*pi**2*r**4)

(1, 1)


3*Q**2*B(r)/(32*pi**2*r**4)

(2, 2)


Q**2/(32*pi**2*r**2)

(3, 3)


Q**2*sin(theta)**2/(32*pi**2*r**2)

Now, print the non-zero components of T:

In [12]:
k = sympy.Symbol("k")
ricci_ = geometry[7]
for i in range(4):
    print(f"({i}, {i})")
    display(sympy.Eq(ricci_[i, i], -k * T_[i, i]))

(0, 0)


Eq(-Derivative(A(r), (r, 2))/(2*B(r)) + Derivative(A(r), r)*Derivative(B(r), r)/(4*B(r)**2) + Derivative(A(r), r)**2/(4*A(r)*B(r)) - Derivative(A(r), r)/(r*B(r)), 3*Q**2*k*A(r)/(32*pi**2*r**4))

(1, 1)


Eq(Derivative(A(r), (r, 2))/(2*A(r)) - Derivative(A(r), r)*Derivative(B(r), r)/(4*A(r)*B(r)) - Derivative(A(r), r)**2/(4*A(r)**2) - Derivative(B(r), r)/(r*B(r)), -3*Q**2*k*B(r)/(32*pi**2*r**4))

(2, 2)


Eq(-r*Derivative(B(r), r)/(2*B(r)**2) + r*Derivative(A(r), r)/(2*A(r)*B(r)) - 1 + 1/B(r), -Q**2*k/(32*pi**2*r**2))

(3, 3)


Eq(-r*sin(theta)**2*Derivative(B(r), r)/(2*B(r)**2) + r*sin(theta)**2*Derivative(A(r), r)/(2*A(r)*B(r)) - (B(r) - 1)*sin(theta)**2/B(r), -Q**2*k*sin(theta)**2/(32*pi**2*r**2))

e)



Notice that taking 
$$
B R_{00} + A R_{11}
$$
vastly simplifies our expression, we get

In [13]:
LHS = sympy.simplify(B * ricci_[0, 0] + A * ricci_[1,1])
RHS = sympy.simplify(-k * (B * T_[0,0] + A * T_[1,1]))

In [14]:
RHS

0

So solving LHS is simple.

In [15]:
sympy.Eq(LHS, 0)

Eq(-A(r)*Derivative(B(r), r)/(r*B(r)) - Derivative(A(r), r)/r, 0)

f)



In [16]:
T_[2, 2]

Q**2/(32*pi**2*r**2)

In [17]:
LHS.subs(A, 1/B)

-Derivative(1/B(r), r)/r - Derivative(B(r), r)/(r*B(r)**2)

In [18]:
LHS2 = sympy.simplify(ricci_[2,2].subs(A, 1/B))

In [19]:
RHS2 = -k * T_[2,2].subs(A, 1/B)

In [20]:
sols = sympy.dsolve(sympy.Eq(LHS2, RHS2))

In [21]:
display(LHS2)
display(RHS2)

-r*Derivative(B(r), r)/B(r)**2 - 1 + 1/B(r)

-Q**2*k/(32*pi**2*r**2)

In [22]:
# B
display(sols.rhs)

# A
display(sympy.simplify(1/sols.rhs))

32*pi**2*r**2/(C1*r + Q**2*k + 32*pi**2*r**2)

C1/(32*pi**2*r) + Q**2*k/(32*pi**2*r**2) + 1

In [23]:
G, M = sympy.symbols("G M", real=True, positive=True)
sympy.simplify(sols.subs("C1", -64*sympy.pi**2*G*M).subs(k, 8*sympy.pi*G))

Eq(B(r), 4*pi*r**2/(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2))

In [24]:
# A
A1 = sympy.simplify((1/sols.subs("C1", -64*sympy.pi**2*G*M).rhs).subs(k, 8*sympy.pi*G))
A1

-2*G*M/r + G*Q**2/(4*pi*r**2) + 1

In [25]:
E

Q*sqrt(A(r)*B(r))/(4*pi*r**2)

In [26]:
sympy.Rational(8, 32)

1/4

In [27]:
displayNonZero(T)

(0, 0)


-3*Q**2/(32*pi**2*r**4*A(r))

(1, 1)


3*Q**2/(32*pi**2*r**4*B(r))

(2, 2)


Q**2/(32*pi**2*r**6)

(3, 3)


Q**2/(32*pi**2*r**6*sin(theta)**2)

In [28]:
sympy.simplify(ricci_[2,2].subs(A, 1/B))

-r*Derivative(B(r), r)/B(r)**2 - 1 + 1/B(r)

In [29]:
sols

Eq(B(r), 32*pi**2*r**2/(C1*r + Q**2*k + 32*pi**2*r**2))

# Finding the singularities

In [30]:
for sol in sympy.solve(A1, r):
    display(sympy.simplify(sol))

-sqrt(G)*sqrt(4*pi*G*M**2 - Q**2)/(2*sqrt(pi)) + G*M

sqrt(G)*sqrt(4*pi*G*M**2 - Q**2)/(2*sqrt(pi)) + G*M

In [31]:
sol1, sol2 = sympy.solve(A1, r)
myexpr = G*M*(1-sympy.sqrt(1 - Q**2 / (G*M**2 * 4 * sympy.pi)))
sympy.simplify(sol1 - myexpr)

0

In [32]:
myexpr

G*M*(1 - sqrt(1 - Q**2/(4*pi*G*M**2)))

In [91]:
riemann = geometry[5]
riemann_ = geometry[6]

d = geometry[0].size
kretScal = 0
for i in range(d):
    for j in range(d):
        for k in range(d):
            for l in range(d):
                kretScal += sympy.simplify((riemann[i, j, k, l] * riemann_[i, j, k, l]).subs(A, A1).subs(B, 1/A1))
kretScal = sympy.simplify(kretScal)

We have two singularities arising from the zero points of A.
Let us now consider them closely.

In [92]:
# myexpr1 and myexpr two correspond to sol1 and sol2 respectively.
myexpr1 = G*M*(1-sympy.sqrt(1 - Q**2 / (G*M**2 * 4 * sympy.pi)))
myexpr2 = G*M*(1+sympy.sqrt(1 - Q**2 / (G*M**2 * 4 * sympy.pi)))

It is `myexpr2` we primarily expect to be a coordinate singularity (and not a physical one), as this reduces to the Schwarzschild radius as the charge goes to zero.

In [93]:
kretScal

(16*pi**2*G**2*r**8*(8*pi*M*r - Q**2)**2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**2*sin(theta)**2 + G**2*(4*pi*M*r - Q**2)**2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**4 - 64*pi**3*r**8*(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2 + 256*pi**4*r**8*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2 + 4*pi*r**4*(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**3*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2 + 64*pi**3*r**4*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)*(-24*pi*G*M*r + 3*G*Q**2 + 16*pi*r**2 + 16*pi*r*(G*M - r))**2 - 4*pi*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**3*(-24*pi*G*M*r + 3*G*Q**2 + 16*pi*r**2 + 16*pi*r*(G*M - r))**2)/(128*pi**4*r**10*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**2)

In [94]:
denominator = 128*sympy.pi**4 * r**10 * (-8 * sympy.pi * G * M * r + G * Q**2 + 4 * sympy.pi * r**2)**2

In [103]:
num_terms = (kretScal * denominator).as_terms()

In [107]:
num_terms[0][0]

(-4*pi*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**3*(-24*pi*G*M*r + 3*G*Q**2 + 16*pi*r**2 + 16*pi*r*(G*M - r))**2,
 ((-12.566370614359172, 0.0), (0, 0, 0, 0, 0, 0, 3, 2, 0, 0), ()))

In [120]:
for term in (kretScal * denominator).as_ordered_terms():
    display(sympy.simplify(term / denominator))

G**2*(8*pi*M*r - Q**2)**2*(cos(theta)**2 - 2)*sin(theta)**2/(8*pi**2*r**2)

G**2*(4*pi*M*r - Q**2)**2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**2/(128*pi**4*r**10)

-(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2/(2*pi*r**2*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2))

2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2/(r**2*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**2)

(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2/(32*pi**3*r**6)

G**2*(64*pi**2*M**2*r**2 - 48*pi*M*Q**2*r + 9*Q**4)/(2*pi*r**6*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2))

(8*pi*G*M*r - G*Q**2 - 4*pi*r**2)*(-24*pi*G*M*r + 3*G*Q**2 + 16*pi*r**2 + 16*pi*r*(G*M - r))**2/(32*pi**3*r**10)

In [137]:
for i, term in enumerate((kretScal * denominator).as_ordered_terms()):
    if i in [2,3,5]:
        display(sympy.trigsimp(
            sympy.simplify(
            (term / denominator)
            )
        ))

-(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2/(2*pi*r**2*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2))

2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2/(r**2*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**2)

G**2*(64*pi**2*M**2*r**2 - 48*pi*M*Q**2*r + 9*Q**4)/(2*pi*r**6*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2))

In [140]:
sympy.expand_trig(kretScal * denominator)

16*pi**2*G**2*r**8*(8*pi*M*r - Q**2)**2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**2*sin(theta)**2 + G**2*(4*pi*M*r - Q**2)**2*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**4 - 64*pi**3*r**8*(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2 + 256*pi**4*r**8*(cos(theta)**2 - 2)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2 + 4*pi*r**4*(sin(theta)**4 + 1)*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**3*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2 + 4*pi*r*(G*M - r))**2 + 64*pi**3*r**4*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)*(-24*pi*G*M*r + 3*G*Q**2 + 16*pi*r**2 + 16*pi*r*(G*M - r))**2 - 4*pi*(-8*pi*G*M*r + G*Q**2 + 4*pi*r**2)**3*(-24*pi*G*M*r + 3*G*Q**2 + 16*pi*r**2 + 16*pi*r*(G*M - r))**2